In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture


### Data

In [ ]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

In [ ]:
test.head()

In [ ]:
train.head()

In [ ]:
val=test["class"]
test=test.drop("class", axis=1)

## Choosing number of clusters and covariance

In [ ]:
def count_clustering_scores(X, k,  score_fun,cov):   
    model_instance = GaussianMixture(n_components=k,covariance_type=cov)
    labels = model_instance.fit_predict(X)
    return score_fun(X, labels)

In [ ]:
from sklearn.metrics import silhouette_score
for cov in ['full', 'tied', 'diag', 'spherical']:
    vec=[]
    vec1=[]
    for i in range(10):
        vec.append(count_clustering_scores(train, i+2, silhouette_score,cov))
        vec1.append(i+2)
    plt.plot(vec1,vec)
    plt.title(cov)
    plt.show()

We are choosing 2 clusters and spherical variance

## Threshold

In [ ]:

gmm = GaussianMixture(n_components=2,covariance_type="spherical")
gmm.fit(train)
scores=gmm.score_samples(train)
np.sort(scores)

In [ ]:
minimum=np.floor(min(scores))
maximum=np.ceil(max(scores))
print(f"Minimum: {minimum}  , Maximum: {maximum}")

### Outlier search

In [ ]:
test_scores=gmm.score_samples(test)
test_scores

In [ ]:
answer=np.logical_or((test_scores<minimum ) ,( test_scores>maximum)).astype(int)

In [ ]:
ans=pd.DataFrame({"predicted":answer,"real":np.array(val)})
ans

### Scores

In [ ]:
from sklearn.metrics import adjusted_mutual_info_score,f1_score, precision_score, recall_score

In [ ]:
print("precision:", precision_score(val,answer))
print("Recall:", recall_score(val,answer))
print("F1:", f1_score(val,answer))
print("Mutual info:",adjusted_mutual_info_score(val,answer))

### Conclusions

All in all outlier prediction using GMM proved succesful making only one wrong prediction, what makes it valuable tool for such a task.